In [1]:
import sys, os, time
#%load_ext autoreload
#%autoreload 2
import foundation as fd
import foundation.util as util
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import torchvision
import h5py as hf
from itertools import product

import ZODB, ZODB.FileStorage
import ZEO
import neo
import transaction
import persistent
from persistent.list import PersistentList
from persistent.mapping import PersistentMapping
from BTrees import OIBTree, IOBTree, IIBTree
from html.parser import HTMLParser

from sql_wikigraph import create_database, WikiGraph_DB
from extraction import wiki_expand, download_article_html

In [15]:
seed = 'Artificial_intelligence'
seed = 'Zero-knowledge_proof'
seed = 'Cryptography'

In [16]:
links, redirs, _ = wiki_expand(seed)
links, redirs = [[link['href'][6:] for link in group] for group in [links, redirs]]
links, redirs = map(list, map(set, [links, redirs]))
if seed in links:
    links.remove(seed)
len(links), len(redirs)

(417, 83)

In [17]:
# html = download_article_html(seed)
# len(html)

In [5]:
# for link in links:
#     if 'ISO_3166' in link:
#         print(link)

In [6]:
db_path = 'alpha.db'

In [7]:
create_database(db_path)

In [8]:
wiki = WikiGraph_DB(db_path)

Updating working sets (0 entries) ...done: expanded=0, inserted=0, redirs=0


In [21]:
#wiki.cursor.execute('UPDATE articles SET status = 1 WHERE rowid = 1;')

In [19]:
list(wiki.items())

[('Zero-knowledge_proof', 1),
 ('Discrete_logarithm', 0),
 ('Zero-knowledge_password_proof', 0),
 ('G%C3%B6del_Prize', 0),
 ('Computational_indistinguishability', 0),
 ('PubMed_Central', 0),
 ('One-time_pad', 0),
 ('Commitment_scheme', 0),
 ('Cryptographic_hash_function', 0),
 ('Probabilistic_Turing_machine', 0),
 ('Moni_Naor', 0),
 ('Prime_number', 0),
 ('Amit_Sahai', 0),
 ('Shafi_Goldwasser', 0),
 ('Feige%E2%80%93Fiat%E2%80%93Shamir_identification_scheme', 0),
 ('International_Standard_Serial_Number', 0),
 ('Turing_machine', 0),
 ('Interactive_proof_system', 0),
 ('Co-NP', 0),
 ('Moti_Yung', 0),
 ('Digital_object_identifier', 0),
 ('Graph_(discrete_mathematics)', 0),
 ('Cryptography', 1),
 ('Blum_integer', 0),
 ('Oded_Goldreich', 0),
 ('One-way_function', 0),
 ('Jean-Jacques_Quisquater', 0),
 ('Non-interactive_zero-knowledge_proof', 0),
 ('Zyryanka_Airport', 0),
 ('Cryptographic_protocol', 0),
 ('Quadratic_residue', 0),
 ('Shlomo_Moran', 0),
 ('NP_(complexity)', 0),
 ('Hamiltonian_pa

In [18]:
wiki.add(seed, links, redirs)

UPDATE articles SET status = 1 WHERE rowid = ?; 23


In [20]:
len(wiki.expanded), len(wiki.inserted), len(wiki.inserted_redirs)

(2, 458, 94)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [22]:
q = mp.Queue()

In [24]:
q.put(2)

In [25]:
len(q)

TypeError: object of type 'Queue' has no len()

In [32]:
ZEO.client?

In [34]:
client = ZEO.client(addr, read_only=True)

In [44]:
client.blob_dir

In [45]:
from ZEO import ClientStorage

In [ ]:
storage = ClientStorage.ClientStorage(addr, read_only=True)
db = ZODB.DB(storage)
conn = db.open()
root = conn.root

In [26]:
def init_worker(db_addr):
    
    db = 
    
    out = {
        'is_new_article' : lambda x: 0,
        'is_new_redirect' : lambda x: 0,
        
    }
    
    return out

def worker_expand(article_name, is_new_article, is_new_redirect, **other_args):
    links, redirs, _ = wiki_expand(seed)
    links, redirs = [[link['href'][6:] for link in group] for group in [links, redirs]]
    links, redirs = map(list, map(set, [links, redirs]))
    if seed in links:
        links.remove(seed)
        
    links = [link for link in links if is_new_article(link)]
    redirs = [redir for redir in redirs if is_new_redirect(redir)]
    
    return article_name, links, redirs
        
    #len(links), len(redirs)

In [ ]:
creator = Farmer(create_sample,
        shared_args=shared_args, private_args=private_args,
        unique_worker_args=[{'seed':s} for s in range(args.num_workers)],
        volatile_gen=gen_idx(args.sets, args.num),
        init_fn=init_worker, num_workers=args.num_workers)